<a href="https://colab.research.google.com/github/ClaudiaMarano/Anomaly-Detection-and-Prediction/blob/main/networkDenseNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Questo è il task di prediction del progetto di Anomaly-Detection-and-Prediction relativo alla parte di Network. Il metodo su cui si basa questo task è quello basato sull'utilizzo di Dense NN


Pipeline per il preprocessing dei dati
--------------------------------------
IMPORTANTE
--------------------------------------
Ad ogni runtime aggiungere nuovamente i file csv in una cartella chiamata csv, non è possibile caricare tutti i csv poichè troppo pesanti

In [22]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Funzione per controllare differenze nelle feature tra i file CSV
def check_feature_differences(file_paths):
    feature_sets = []

    for file in file_paths:
        try:
            df = pd.read_csv(file, nrows=1)  # Legge solo la prima riga per ottenere le feature
            df.columns = df.columns.str.strip()  # Rimuove spazi dai nomi delle colonne
            feature_sets.append(set(df.columns))
        except Exception as e:
            print(f"Errore durante la lettura del file {file}: {e}")

    # Confronta le feature tra i file
    common_features = set.intersection(*feature_sets)  # Feature comuni a tutti i file
    unique_features = [features - common_features for features in feature_sets]  # Feature uniche per file

    # Output dei risultati
    result = {
        "common_features": common_features,
        "unique_features_per_file": {
            os.path.basename(file): unique for file, unique in zip(file_paths, unique_features)
        }
    }

    return result

# Funzione per unire i file CSV
def merge_csv_files(file_paths):
    dataframes = []

    for file in file_paths:
        try:
            df = pd.read_csv(file)
            df.columns = df.columns.str.strip()  # Rimuove spazi dai nomi delle colonne
            dataframes.append(df)
        except pd.errors.EmptyDataError:
            print(f"File vuoto: {file}")
        except pd.errors.ParserError as e:
            print(f"Errore di parsing nel file {file}: {e}")
        except Exception as e:
            print(f"Errore sconosciuto durante la lettura del file {file}: {e}")

    try:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print(f"Unione completata. Numero totale di righe: {combined_df.shape[0]}")
        return combined_df
    except ValueError as e:
        print(f"Errore durante l'unione dei file: {e}")
        return pd.DataFrame()

# Funzione per analizzare le feature e identificare quelle inutili
def analyze_features(df):
    try:
        # Calcolo del numero di valori unici per ogni colonna
        unique_counts = df.nunique()

        # Identificazione di feature con valori mancanti
        missing_values = df.isna().sum()

        # Identificazione di feature con valori costanti
        constant_features = [col for col in df.columns if unique_counts[col] == 1]

        # Report
        print("Analisi delle feature:")
        print("Numero di valori unici per feature:")
        print(unique_counts)
        print("Valori mancanti per feature:")
        print(missing_values)
        print("Feature con valori costanti:")
        print(constant_features)
    except Exception as e:
        print(f"Errore durante l'analisi delle feature: {e}")

# Funzione per rimuovere la feature 'label'
def remove_label_feature(df):
    try:
        if 'label' in df.columns:
            df = df.drop(columns=['label'])
            print("Feature 'label' rimossa con successo.")
        else:
            print("Feature 'label' non trovata nel dataset.")
        return df
    except Exception as e:
        print(f"Errore durante la rimozione della feature 'label': {e}")
        return df

# Funzione per rimuovere la feature 'modbus_response'
def remove_modbus_response_feature(df):
    try:
        if 'modbus_response' in df.columns:
            df = df.drop(columns=['modbus_response'])
            print("Feature 'modbus_response' rimossa con successo.")
        else:
            print("Feature 'modbus_response' non trovata nel dataset.")
        return df
    except Exception as e:
        print(f"Errore durante la rimozione della feature 'modbus_response': {e}")
        return df


# Funzione per rimuovere la feature 'modbus_fn'
def remove_modbus_fn_feature(df):
    try:
        if 'modbus_fn' in df.columns:
            df = df.drop(columns=['modbus_fn'])
            print("Feature 'modbus_fn' rimossa con successo.")
        else:
            print("Feature 'modbus_fn' non trovata nel dataset.")
        return df
    except Exception as e:
        print(f"Errore durante la rimozione della feature 'modbus_fn': {e}")
        return df


# Funzione per eliminare i valori mancanti dal dataset
def drop_missing_values(df):
    try:
        initial_shape = df.shape
        df = df.dropna()
        print(f"Valori mancanti eliminati. Dimensione iniziale: {initial_shape}, Dimensione dopo eliminazione: {df.shape}.")
        return df
    except Exception as e:
        print(f"Errore durante l'eliminazione dei valori mancanti: {e}")
        return df

# Directory del dataset Network
dataset_path = "./csv"
network_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith('.csv')]

# Controllo delle differenze nelle feature tra i file
feature_differences = check_feature_differences(network_files)

# Output dei risultati
print("Feature comuni a tutti i file:")
print(feature_differences["common_features"])

print("Feature uniche per ciascun file:")
for file, unique_features in feature_differences["unique_features_per_file"].items():
    print(f"File: {file}")
    print(f"Feature uniche: {unique_features}")

# Unione dei file CSV
combined_dataset = merge_csv_files(network_files)

# Analisi delle feature
analyze_features(combined_dataset)

# Rimozione della feature 'label'
combined_dataset = remove_label_feature(combined_dataset)

# Rimozione della feature 'modbus_response'
combined_dataset = remove_modbus_response_feature(combined_dataset)

# Rimozione della feature 'modbus_fn'
combined_dataset = remove_modbus_fn_feature(combined_dataset)

# Eliminazione dei valori mancanti
combined_dataset = drop_missing_values(combined_dataset)

combined_dataset.head()

# Analisi delle feature
analyze_features(combined_dataset)

Feature comuni a tutti i file:
{'sport', 'proto', 'modbus_response', 'label_n', 'Time', 'ip_d', 'dport', 'mac_d', 'flags', 'mac_s', 'n_pkt_dst', 'modbus_fn', 'ip_s', 'label', 'n_pkt_src', 'size'}
Feature uniche per ciascun file:
File: attack_4.csv
Feature uniche: set()
File: attack_2.csv
Feature uniche: set()
File: attack_1.csv
Feature uniche: set()
File: attack_3.csv
Feature uniche: set()
File: normal.csv
Feature uniche: set()
Unione completata. Numero totale di righe: 953689
Analisi delle feature:
Numero di valori unici per feature:
Time               913601
mac_s                   9
mac_d                  11
ip_s                    8
ip_d                    8
sport                2291
dport                2291
proto                   3
flags                   7
size                   30
modbus_fn               5
modbus_response      1566
n_pkt_src              54
n_pkt_dst              54
label_n                 2
label                   4
dtype: int64
Valori mancanti per feature:
T